In [1]:
import numpy as np
import pandas as pd
import random
import torch
import pickle

In [2]:
from torch.utils.tensorboard import SummaryWriter

In [3]:
from PortfolioConstructor import PortfolioConstructor
from ExchnageEnv import MarketEnvironment

In [4]:
if torch.cuda.is_available():
    torch.set_default_device('cuda') 
    torch.get_default_device()
    device = 'cuda'
    
else:
    device = 'cpu'

print(f"device : {device}")

device : cuda


In [5]:
pkl_fpath = '/home/naradaw/dev/Charles_Schwab/data/historical_random_100/2024_10_15/historical_price_seq_2024_10_15_16_22.pkl'

with open(pkl_fpath, 'rb') as f:
    price_sqs_dict = pickle.load(f)

In [6]:
price_sqs_dict[list(price_sqs_dict.keys())[0]].shape

(1174, 61)

In [7]:
symbol_universe = list(price_sqs_dict.keys())
len(symbol_universe)

100

In [8]:
# test_symbol_uni = random.choices(list(price_sqs_dict.keys()), k = 20)


In [9]:
portfolio_constructor = PortfolioConstructor(
    device= device,
    symbol_universe = symbol_universe,
    seq_length = 60,
    multihead_dim = 2,
    num_transformer_layers = 2
)

/home/naradaw/miniconda3/envs/tf-wsl/lib/python3.9/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [10]:
market_env = MarketEnvironment(
    data_path = pkl_fpath,
    holding_period = 1,
    train_test_split= 0.8,
    symbol_universe = symbol_universe,
    device = device
    )

In [11]:
market_env.data.shape

(1174, 100, 61)

# Utility

In [12]:
''' 
sharpe ratio measures the excess return of the portfolio over the 
volatility of it -> risk adjusted performance
'''


def sharp_ratio_(rewards, tran_costs):

	# rewards = [r.detach().cpu().numpy() for r in rewards]
	mean = sum(rewards) / len(rewards)
	At = sum(r - t for r, t in zip(rewards, tran_costs)) / len(rewards)
	vol = sum((r - mean) ** 2 for r in rewards) / len(rewards)
	vol = vol ** 0.5

	return (At - 1e-7) / (vol + 1e-9)

# Train

In [13]:
portfolio_constructor.cuda()
portfolio_constructor.train()
market_env.reset(mode = "train")

In [14]:
portfolio_constructor.parameters()

<generator object Module.parameters at 0x7f090ab825f0>

In [15]:
optimizer = torch.optim.Adam(portfolio_constructor.parameters())

In [16]:
market_env.get_state()

tensor([[149.8100, 150.2500, 145.8600,  ..., 131.0100, 132.2100, 132.8400],
        [186.5200, 183.2800, 188.7600,  ..., 183.9000, 186.3400, 187.0000],
        [219.0800, 214.2200, 217.1900,  ..., 214.1700, 220.3600, 218.3800],
        ...,
        [ 35.9000,  36.3900,  36.8200,  ...,  33.6900,  34.1100,  34.2700],
        [ 88.7900,  86.5700,  88.7800,  ...,  88.0600,  89.1200,  89.2200],
        [152.5200, 149.5800, 155.0300,  ..., 151.1900, 153.3600, 157.7400]],
       device='cuda:0')

# Sandbox

In [27]:
def sharp_ratio_loss_(rewards, tran_costs, allocations):

	# rewards = [r.detach().cpu().numpy() for r in rewards]
	mean = sum(rewards) / len(rewards)
	At = sum(r - t for r, t in zip(rewards, tran_costs)) / len(rewards)
	vol = sum((r - mean) ** 2 for r in rewards) / len(rewards)
	vol = vol ** 0.5

	return (At - 1e-7) / (vol + 1e-9)

In [28]:
training_steps = 500
eval_step = 32
train_step = 32


In [29]:
def evaluate(model, env):
    model.eval()
    is_end = False
    rewards = []
    tran_costs = []
    
    env.reset(mode = "test")
    state = env.get_state()

    while not is_end:
        symbol_idx , allocations = model(state)
        state, reward, is_end, tran_cost = env.step(allocations)

        rewards.append(reward)
        tran_costs.append(tran_cost)

    # print()
    # print("evaluation")
    # print(f"rewards : {rewards}")
    # print(f"tran_costs : {tran_costs}")
    # print()

    sharp_ratio = sharp_ratio_(rewards, tran_costs)
    
    model.train()

    return sharp_ratio

In [30]:
writer = SummaryWriter()

In [31]:
max_reward = -1

for training_step in range(training_steps):
    is_end = False
    rewards = []
    tran_costs = []
    nlls = []

    market_env.reset(mode = "train", transaction_cost= 1e-7)
    state = market_env.get_state()

    while not is_end:
        symbol_idx, allocations = portfolio_constructor(state)
        state, reward, is_end, tran_cost = market_env.step(allocations)

        rewards.append(reward)
        tran_costs.append(tran_cost)
        mask_tensor = torch.tensor([1 if i in symbol_idx.cpu().numpy() else 0 for i in range(allocations.shape[0])]).type(torch.FloatTensor).cuda()
        # nlls.append(torch.log(allocations.abs() / 2 + 1e-9) * mask_tensor)
        nlls.append(-(torch.log(allocations.abs() / 2 + 1e-9) * mask_tensor))

    
    sharp_ratio = sharp_ratio_(rewards, tran_costs)
    loss = -sharp_ratio * sum([e.sum() for e in nlls])

    loss.backward(retain_graph=True)

    if (training_step + 1) % train_step == 0:

        print("-------------------------------------")
        print("training model --")
        print('Step {}: last loss = {:.5f}\r'.format(training_step, loss), end='')
        print()
        writer.add_scalar("Loss/train", sharp_ratio, training_step)
        # pprint([(n, e.grad) for n, e in model.named_parameters()])
        optimizer.step()
        optimizer.zero_grad()
        count = 0
        
    if (training_step + 1) % eval_step == 0:
        print("eval step --")
        with torch.no_grad():
            reward_val = evaluate(portfolio_constructor, market_env)
            print('Step {}: val_rewards = {}'.format(training_step, reward_val))
            writer.add_scalar("eval_sharpe/train", reward_val, training_step)

            if max_reward < reward_val:
                max_reward = reward_val

                print("*** found better model ***")
            print()
                # torch.save(portfolio_constructor.state_dict(), model_path)




-------------------------------------
training model --
Step 31: last loss = -87.33910
eval step --
Step 31: val_rewards = -0.0032945660973629316
*** found better model ***

-------------------------------------
training model --
Step 63: last loss = -105.36764
eval step --
Step 63: val_rewards = 0.03494951313978346
*** found better model ***

-------------------------------------
training model --
Step 95: last loss = -59.76510
eval step --
Step 95: val_rewards = 0.7129135422836762
*** found better model ***

-------------------------------------
training model --
Step 127: last loss = -73.90375
eval step --
Step 127: val_rewards = 0.7437895064767789
*** found better model ***

-------------------------------------
training model --
Step 159: last loss = -41.08833
eval step --
Step 159: val_rewards = 0.6969566369259936

-------------------------------------
training model --
Step 191: last loss = 134.43474
eval step --
Step 191: val_rewards = 0.6306652273859508

----------------------

In [32]:
random_state = market_env.get_random_state()
random_state.detach().cpu().numpy()

array([[114.29, 114.03, 114.48, ..., 126.04, 127.31, 129.41],
       [148.21, 149.83, 149.36, ..., 176.96, 177.38, 182.95],
       [217.1 , 218.18, 210.12, ..., 253.89, 260.55, 272.77],
       ...,
       [ 42.01,  41.65,  41.64, ...,  35.43,  35.26,  35.24],
       [ 88.24,  84.98,  84.82, ...,  95.76,  95.81,  98.74],
       [142.64, 140.48, 140.27, ..., 162.5 , 163.19, 167.59]],
      dtype=float32)

In [33]:
portfolio_constructor(random_state)

(tensor([32, 74, 10, 79, 18, 83, 22, 23, 87, 26], device='cuda:0'),
 tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.1000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.1000, 0.0000, 0.0000, 0.0000, 0.1000, 0.1000, 0.0000, 0.0000, 0.1000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.1000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1000, 0.0000,
         0.0000, 0.0000, 0.1000, 0.0000, 0.0000, 0.0000, 0.1000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000], device='cuda:0', grad_f

In [34]:
writer.add_graph(portfolio_constructor, random_state.detach())

In [35]:
writer.flush()
writer.close()

In [ ]:
!tensorboard --logdir=runs

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.18.0 at http://localhost:6006/ (Press CTRL+C to quit)
